<a href="https://colab.research.google.com/github/joshIsac/LargeLanguageModel/blob/main/2348523_LLM__lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install datasets
!pip install tensorflow

In [93]:
#loading the imdb datasets using tensorflow
import torch as torch
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

In [94]:
#loading the datasets of imdb
from datasets import load_dataset
imdb_reviews=load_dataset('imdb')

In [95]:
import random
postive = [x for x in imdb_reviews['train'] if x['label'] == 1][:2500]
negative = [x for x in imdb_reviews['train'] if x['label'] == 0][:2500]
train_dataset = postive + negative
random.shuffle(train_dataset)
positive_test = [x for x in imdb_reviews['test'] if x['label'] == 1][:2500]
negative_test = [x for x in imdb_reviews['test'] if x['label'] == 0][:2500]
test_dataset = positive_test + negative_test
random.shuffle(test_dataset)

In [96]:
#loading the BERT model
# Load model directly
from transformers import BertTokenizer, TFBertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [97]:
#defining train ans test data
train_texts = [x['text'] for x in train_dataset]
train_labels = [x['label'] for x in train_dataset]
test_texts = [x['text'] for x in test_dataset]
test_labels = [x['label'] for x in test_dataset]

In [98]:
#tokenizing
train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True,max_length=128)

In [99]:
#Mapping the train and test
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))


train_dataset = train_dataset.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

In [101]:
import tensorflow as tf

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [102]:
#fit the model
model.fit(train_dataset, epochs=1, validation_data=test_dataset)

313/313 [==============================] - 229s 536ms/step - loss: 0.7242 - accuracy: 0.4998 - val_loss: 0.6931 - val_accuracy: 0.5000


In [103]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.2f}")

313/313 [==============================] - 45s 144ms/step - loss: 0.6931 - accuracy: 0.5000
Test Accuracy: 0.50


In [104]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    outputs = model(inputs)
    logits = outputs.logits
    predictions = tf.nn.softmax(logits, axis=-1)
    sentiment = tf.argmax(predictions, axis=1).numpy()[0]
    return sentiment

In [106]:
text = input("Enter text for sentiment analysis: ")
sentiment = predict_sentiment(text)
sentiment_label = "Positive" if sentiment == 1 else "Negative"
print(f"Sentiment: {sentiment_label}")


Enter text for sentiment analysis: amazing moves by the actor
Sentiment: Positive
